# Crafter

In [1]:
import os
import csv
import json
import torch
import numpy as np
from transformers import CLIPProcessor, CLIPModel
from generate_embeddings import generate_embeddings
from openai import OpenAI

d:\Computer_Download\anaconda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## 1. Help Function

### 1.1 Model Calling

In [3]:
def kimi(question, history=None):
	client = OpenAI(
		api_key = "sk-DhVW0coT3r2Kj4hpqX9Uy7p0NRgj6ETBG9WfB1cHQLoDrOgO",
		base_url = "https://api.moonshot.cn/v1",
	)

	messages = [
			{
			"role": "user",
			"content": question
			}
		]
	
	if history is not None:
		messages = history + messages

	completion = client.chat.completions.create(
		model = "moonshot-v1-8k",
		messages = messages,
		temperature = 0.3,
	)

	history = messages + [
		{
			"role": "assistant",
			"content": completion.choices[0].message.content
		}
	]

	return completion.choices[0].message.content, history

In [4]:
def deepseek(question, history=None, model="r1"):
	client = OpenAI(
		base_url="https://openrouter.ai/api/v1",
		api_key="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",
	)

	if model == "v3":
		model_name = "deepseek/deepseek-v3-base:free"
	elif model == "r1":
		model_name = "deepseek/deepseek-r1:free"
	else:
		return "Invalid model name"

	messages = [
			{
			"role": "user",
			"content": question
			}
		]
	
	if history is not None:
		messages = history + messages

	completion = client.chat.completions.create(
		extra_headers={},
		extra_body={},
		model=model_name,
		messages=messages
	)

	history = messages + [
		{
			"role": "assistant",
			"content": completion.choices[0].message.content
		}
	]

	return completion.choices[0].message.content, history

### 1.2 JSON Operate

In [5]:
def load_json(filepath):
	with open(filepath, 'r', encoding='utf-8') as file:
		cad = json.load(file)
	return cad

In [6]:
def available_path(dirpath='./outputs'):
    idx = 0
    while os.path.exists(dirpath + f"generate{str(idx)}.json"):
        idx += 1
    return dirpath + f"generate{str(idx)}.json"

In [7]:
def save_json(json_dic, filepath):
    with open(filepath, "w") as file:
    	json.dump(json_dic, file, indent=4)

In [8]:
def json_generate(prompt, model_call=deepseek, limit=3):
    count = 0
    history = []
    while count < limit:
        gene_s, history = model_call(prompt, history)
        if gene_s.count('```') >= 2:
            start = gene_s.find('```json\n')
            gene_s = gene_s[start+8:]
            end = gene_s.find('```')
            gene_json = json.loads(gene_s[:end])
            return gene_json, True
        else:
            count += 1
            print("Go on...")
            prompt = "继续生成"
    return gene_s, False

### 1.3 CLIP

In [9]:
def load_captions():
    captions = {}
    pth_paths = {}  # 独立字典
    captions_file = 'data.csv'
    if not os.path.exists(captions_file):
        return captions, pth_paths
    with open(captions_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 3:
                image_path = row[0].strip('"')
                captions[image_path] = row[1]
                pth_paths[image_path] = row[2].strip()  # 独立存储
    return captions, pth_paths  # 返回两个独立字典

In [10]:
model_name = "laion/CLIP-ViT-L-14-DataComp.XL-s13B-b90K"
clip_model = CLIPModel.from_pretrained(model_name).to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained(model_name)
captions_dict, pth_paths_dict = load_captions()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [11]:
'''
这里是原来关键词查top图片的那个
参数说明：
    keyword是一段文本， 
    n_images是要输出top多少的，
    use_captions是要不要用caption
返回值：
    top_images:是一个数组里面有n_images个字典，每个字典有：
        "filename":图片文件名
        "similarity": 相似度数值
        "path":图片路径
        "caption": 描述字段
        "pth_path": CAD文件路径
    error就是error
'''
def top_images_view(keyword, n_images, use_captions):
    error = None
    top_images = []
    use_captions = False

    if not keyword:
        error = "关键词为空"
    elif not n_images:
        error = "无图片数量"
    else:
        try:
            if n_images < 1 or n_images > 1000:
                error = "数量不在1-1000"
            else:
                image_paths, _, similarities = generate_embeddings(
                    clip_model, clip_processor, keyword, captions_dict, use_captions
                )
                results = [
                    {
                    "filename": os.path.basename(path),
                    "similarity": sim,
                    "path": path,
                    "caption": captions_dict.get(path, "No caption") if use_captions else "Captions disabled",
                    "pth_path": pth_paths_dict.get(path, "No pth path")  # 始终包含
                    }
                    for path, sim in zip(image_paths, similarities)
                ]
                top_images = sorted(results, key=lambda x: x["similarity"], reverse=True)[:n_images]
        except ValueError:
            error = "非有效的正整数"
        except Exception as e:
            error = f"计算相似度失败: {e}"

    return top_images, error

## 2. Ways

### 2.1 Direct Generate

In [12]:
def whole_generate(desire):
	simple = load_json('./examples/simple.json')
	simple_s = json.dumps(simple, indent=4)
	prompt = simple_s + f"请按照如上给出的minimal_json格式生成一个{desire}的CAD程序。" + \
		"严格按照示例，要求各个部分为“part_1/2/3”命名，part中包含'coordinate_system'，'extrusion'，以及'sketch'字段。" + \
		"'sketch'字段中用face描述面，face中用loop描述构成面的线，loop中只支持arc, line, circle三种。" + \
		"这里的点全部用'Start Point'，'Mid Point'，'End Point'字段描述，line需要start和end，arc需要start, mid end，对应的是B-ref曲线。" + \
		"circle需要'Center'和'Radius'两个子字段"
	return json_generate(prompt)

### 2.2 Component & Join

#### 2.2.1 Divide

In [13]:
def gene_division(desire):
    
	pass

In [ ]:
def get_division(desire):
	example = load_json('./examples/get_div.json')
	example_s = json.dumps(example, indent=4)
	prompt = example_s + f"请按照上述的json格式将一个'{desire}'的复杂图形拆分成小组件的组合。" +\
		"整体结构与各个字段严格按照示例，整体分为name, parts和positional relationship三个部分。" +\
		"parts中按照part_1/2/3进行编号，每个子结构中有id, type, description, params, position等参数" +\
		"id是唯一标识符，type只有cuboid和cylinder两种，前者params长为3，分别描述长宽高；后者params长为2，对应半径与高度。postion长为3，描述该物体相对于坐标原点的偏移量" +\
		"positional relationship为一个列表，每个元素是一个字典，描述的是各个物体的相对位置关系。" +\
		"from和to对应的是物体的id，表示这里描述的是to物体相对于from物体的位置。relationship分为三个维度，每个维度通过一个长为2的字符串列表描述" +\
		"x轴由left/right/align表示相对位置; y轴由front/back/align表示; z轴由up/down/align表示. 第二个维度用intersect和seperate表示是否相交" +\
		f"请按照上述的json格式将一个'{desire}'的复杂图形拆分成小组件的组合。"
	return json_generate(prompt)

#### 2.2.2 Get

In [15]:
def component_generate(shape):
	example = load_json('./basics/cuboid.json')
	example_s = json.dumps(example, indent=4)
	prompt = example_s + f"请按照上述minimal_json的格式生成一个'{shape}'的CAD程序。" + \
		"严格按照示例，要求各个部分为“part_1/2/3”命名，part中包含'coordinate_system'，'extrusion'，以及'sketch'字段。" + \
		"'sketch'字段中用face描述面，face中用loop描述构成面的线，loop中只支持arc, line, circle三种。" + \
		"这里的点全部用'Start Point'，'Mid Point'，'End Point'字段描述，line需要start和end，arc需要start, mid end，对应的是B-ref曲线。" + \
		"circle需要'Center'和'Radius'两个子字段"
	return json_generate(prompt, deepseek)

In [ ]:
def component_search(shape, num=1, caption=False):
	top_images, _ = top_images_view(shape, num, caption)
	cads = []
	for item in top_images:
		uid = item['filename'][:8]
		filepath = f"../../dataset/{uid[:4]}/{uid}/minimal_json/{uid}_merged_vlm.json"
		cad = load_json(filepath)
		cads.append(json.loads(cad))
		# TODO
		break
	return cads[0], True

In [17]:
def cuboid_set(cuboid, params):
	l = float(params['length'])
	w = float(params['width'])
	h = float(params['height'])

	loop = cuboid['part_1']['sketch']['face_1']['loop_1']

	line_1 = loop['line_1']
	line_1['Start Point'] = [0.0, 0.0]
	line_1['End Point'] = [l, 0.0]

	line_2 = loop['line_2']
	line_2['Start Point'] = [l, 0.0]
	line_2['End Point'] = [l, w]

	line_3 = loop['line_3']
	line_3['Start Point'] = [l, w]
	line_3['End Point'] = [0.0, w]

	line_4 = loop['line_4']
	line_4['Start Point'] = [0.0, w]
	line_4['End Point'] = [0.0, 0.0]

	cuboid['extrusion']['extrude_depth_towards_normal'] = h

	des = cuboid['description']
	des['length'] = l
	des['width'] = w
	des['height'] = h
	des['shape'] = f"A rectangular prism with length {l}, width {w} and height {h}"

def cylinder_set(cylinder, params):
	radius = float(params['radius'])
	h = float(params['height'])

	circle = cylinder['part_1']['sketch']['face_1']['circle_1']
	circle['Center'] = [radius, radius]
	circle['Radius'] = radius

	cylinder['extrusion']['extrude_depth_towards_normal'] = h

	des = cylinder['description']
	des['length'] = radius
	des['width'] = radius
	des['height'] = h
	des['shape'] = f'A circular cylinder with diameter {radius} and height {h}'

def component_get(type, params):
	cad_s = load_json(f"./basics/{type}.json")
	cad_json = json.loads(cad_s)
	if type == "cuboid":
		cuboid_set(cad_json, params)		
	elif type == "cylinder":
		cylinder_set(cad_json, params)
	return [cad_json], True

### 2.2.3 Join

In [ ]:
def params_join(parts_dic):
    parts = parts_dic['parts']
    for part_id in parts:
        part = parts[part_id]
        
    pass

## 3. Interact

In [19]:
gene_json, res = get_division("a boy with round head and cubic body, arms and legs")

Go on...
Go on...


In [21]:
print(json.dumps(gene_json, indent=4))

{
    "name": "a boy with round head and cubic body, arms and legs",
    "parts": {
        "part_1": {
            "id": "1",
            "type": "cylinder",
            "description": "head",
            "params": [
                0.15,
                0.2
            ],
            "position": [
                0.0,
                0.0,
                0.35
            ]
        },
        "part_2": {
            "id": "2",
            "type": "cuboid",
            "description": "body",
            "params": [
                0.3,
                0.2,
                0.5
            ],
            "position": [
                0.0,
                0.0,
                0.0
            ]
        },
        "part_3": {
            "id": "3",
            "type": "cuboid",
            "description": "left arm",
            "params": [
                0.1,
                0.1,
                0.4
            ],
            "position": [
                -0.2,
                0.0,
       